<a href="https://colab.research.google.com/github/Jieoi/heart_mur/blob/main/lora_heartmur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os, io, json

def clean_notebook(path):
    path = os.path.abspath(path)
    nb = json.load(io.open(path, "r", encoding="utf-8"))
    if "metadata" in nb and "widgets" in nb["metadata"]:
        del nb["metadata"]["widgets"]
        json.dump(nb, io.open(path, "w", encoding="utf-8"), ensure_ascii=False, indent=2)
        print("Cleaned:", path)
    else:
        print("No widgets metadata found:", path)

clean_notebook("/content/drive/MyDrive/Colab Notebooks/lora_heartmur.ipynb")

Cleaned: /content/drive/MyDrive/Colab Notebooks/lora_heartmur.ipynb


In [ ]:
# Ensure GPU is enabled: Runtime → Change runtime type → GPU
!pip -q install --upgrade pip setuptools wheel packaging

# Match Colab CUDA (12.1)
!pip -q install --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1

# Core libraries
!pip -q install diffusers>=0.31.0 transformers peft accelerate \
  librosa soundfile torchmetrics datasets einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
!pip -q install torchsde # dependency for CosineDPMSolverMultistepScheduler

In [ ]:
import os, glob, pandas as pd
from pathlib import Path

DATA_ROOT   = "/content/drive/MyDrive/datasets/physionet"
CIRCOR_VER  = "1.0.3"
CIRCOR_ROOT = f"{DATA_ROOT}/physionet.org/files/circor-heart-sound/{CIRCOR_VER}"

# load the latest split directory produced previously
SPLIT_DIRS = sorted(glob.glob(f"{CIRCOR_ROOT}/splits/patient_splits_*"))
assert SPLIT_DIRS, "No split directories found under CIRCOR_ROOT/splits. Create patient-level splits first."
SPLIT_DIR = SPLIT_DIRS[-1]
print("Using split dir:", SPLIT_DIR)

Using split dir: /content/drive/MyDrive/datasets/physionet/physionet.org/files/circor-heart-sound/1.0.3/splits/patient_splits_20250926_084757


In [ ]:
# checking
train_csv = os.path.join(SPLIT_DIR, "train_manifest.csv")
val_csv   = os.path.join(SPLIT_DIR, "val_manifest.csv")
test_csv  = os.path.join(SPLIT_DIR, "test_manifest.csv")
for p in (train_csv, val_csv, test_csv):
    assert os.path.exists(p), f"Missing manifest: {p}"

In [ ]:
train_df = pd.read_csv(train_csv)
val_df   = pd.read_csv(val_csv)

print("Train shape:", train_df.shape, "| Val shape:", val_df.shape)
print("Train outcome counts:\n", train_df['outcome'].value_counts())
print("Val outcome counts:\n",   val_df['outcome'].value_counts())


Train shape: (2220, 5) | Val shape: (469, 5)
Train outcome counts:
 outcome
Normal      1145
Abnormal    1075
Name: count, dtype: int64
Val outcome counts:
 outcome
Normal      250
Abnormal    219
Name: count, dtype: int64


In [ ]:
import torch
print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
else:
    print("WARNING: No GPU available. Enable GPU in Runtime.")

Torch: 2.5.1+cu121 | CUDA: 12.1 | GPUs: 1
CUDA device: Tesla T4


In [ ]:
from diffusers import StableAudioPipeline
import torch

MODEL_REPO = "stabilityai/stable-audio-open-1.0"

In [ ]:
from huggingface_hub import login, whoami
login()
print(whoami())

{'type': 'user', 'id': '675d79f5f2b7722d135416cb', 'name': 'jiejiexin', 'fullname': 'Jie Xin', 'email': 'xinnnjie@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/WZwtoT_XqasHbqaqCqyTE.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'heart_lore', 'role': 'read', 'createdAt': '2025-09-26T13:10:38.847Z'}}}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
pipe = StableAudioPipeline.from_pretrained(
    MODEL_REPO, torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
# fix sample rate
SR_MODEL = getattr(pipe, "sample_rate", 44100)
try:
    AUDIO_CHANNELS = getattr(pipe, "audio_channels", 2)
except Exception:
    AUDIO_CHANNELS = 2

In [ ]:
print("Stable Audio Open — sample_rate:", SR_MODEL, "| channels:", AUDIO_CHANNELS)
assert SR_MODEL == 44100, f"Expected 44.1 kHz; got {SR_MODEL}. Adjust code if model SR differs."

Stable Audio Open — sample_rate: 44100 | channels: 2


In [ ]:
import numpy as np, soundfile as sf, librosa
from typing import List, Dict

In [ ]:
SEG_SEC = 6.0    # window length
HOP_SEC = 3.0    # hop length (50% overlap)

In [ ]:
def build_segment_index(manifest_df: pd.DataFrame, prefer_local: bool=False) -> List[Dict]:
    """
    Build a list of segment descriptors without decoding audio.
    Each item: {path, start, frames, sr_src, site, outcome, murmur, patient_id, label}
    label: 1 for Abnormal, 0 for Normal (for sampler)
    """
    items = []
    for _, r in manifest_df.iterrows():
        wav = r["audio"]
        if prefer_local and wav.startswith(CIRCOR_ROOT):
            wav = wav.replace(f"{CIRCOR_ROOT}/training_data", "/content/training_data")
        try:
            info = sf.info(wav)
        except Exception as e:
            print("Skipping (cannot read):", wav, e)
            continue
        sr = info.samplerate
        total_frames = info.frames
        win = int(SEG_SEC * sr)
        hop = int(HOP_SEC * sr)
        if total_frames < win:
            continue
        starts = list(range(0, total_frames - win + 1, hop))
        label = 1 if str(r["outcome"]).strip() == "Abnormal" else 0
        for s in starts:
            items.append({
                "path": wav, "start": s, "frames": win, "sr_src": sr,
                "site": r["site"], "outcome": r["outcome"], "murmur": r.get("murmur", "Unknown"),
                "patient_id": r["patient_id"], "label": label,
            })
    return items

In [ ]:
print("Indexing train segments...")
train_index = build_segment_index(train_df, prefer_local=False)
print("Indexing val segments...")
val_index   = build_segment_index(val_df,   prefer_local=False)

print(f"Train segments: {len(train_index)} | Val segments: {len(val_index)}")

Indexing train segments...
Indexing val segments...
Train segments: 13512 | Val segments: 2904


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [ ]:
def make_prompt(site, outcome, murmur):
    parts = [f"site={str(site)}", f"outcome={str(outcome).lower()}"]
    parts.append("murmur=present" if str(murmur)=="Present" else "murmur=absent")
    return "; ".join(parts)

In [ ]:
class PCGSegmentDataset(Dataset):
    def __init__(self, seg_index):
        self.items = seg_index
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        it = self.items[i]
        path, start, frames, sr_src = it["path"], it["start"], it["frames"], it["sr_src"]
        x, sr = sf.read(path, start=start, frames=frames, dtype="float32", always_2d=False)
        if x.ndim > 1:
            x = np.mean(x, axis=1).astype(np.float32)  # collapse to mono
        # Resample 4 kHz → 44.1 kHz
        if sr != SR_MODEL:
            x = librosa.resample(x, orig_sr=sr, target_sr=SR_MODEL, res_type="soxr_vhq").astype(np.float32)
        # Peak normalization (clip-safe)
        peak = float(np.max(np.abs(x)) + 1e-8)
        x = (x / peak).astype(np.float32)
        # Duplicate to stereo
        x_st = np.stack([x, x], axis=0)  # [2, T]
        prompt = make_prompt(it["site"], it["outcome"], it["murmur"])
        return {
            "audio": torch.from_numpy(x_st),   # [2, T] float32 in [-1, 1]
            "prompt": prompt,
            "label": it["label"],              # 1 abnormal / 0 normal (for sampling only)
            "patient_id": str(it["patient_id"]),
        }


In [ ]:
train_ds = PCGSegmentDataset(train_index)
val_ds   = PCGSegmentDataset(val_index)

In [ ]:
def make_balanced_sampler(index):
    labels = [it["label"] for it in index]
    pos = sum(labels)
    neg = len(labels) - pos
    w_pos = 0.5 / max(1, pos)
    w_neg = 0.5 / max(1, neg)
    weights = [w_pos if y==1 else w_neg for y in labels]
    return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

In [ ]:
BATCH = 4  # T4
train_loader = DataLoader(train_ds, batch_size=BATCH, sampler=make_balanced_sampler(train_index),
                          num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                          num_workers=2, pin_memory=True)

In [ ]:
batch = next(iter(train_loader))
print("Batch audio shape:", batch["audio"].shape, "| example prompt:", batch["prompt"][0])

Batch audio shape: torch.Size([4, 2, 264600]) | example prompt: site=PV; outcome=abnormal; murmur=absent


In [ ]:
text_encoder = pipe.text_encoder
tokenizer    = pipe.tokenizer
vae          = pipe.vae
transformer  = pipe.transformer
scheduler    = pipe.scheduler

for p in text_encoder.parameters(): p.requires_grad = False
for p in vae.parameters():          p.requires_grad = False
for p in transformer.parameters():  p.requires_grad = True  # LoRA will narrow this to adapters only

In [ ]:
import inspect, torch.nn as nn
from diffusers.models.attention_processor import LoRAAttnProcessor
from diffusers.loaders import AttnProcsLayers

In [ ]:
for p in transformer.parameters():
    p.requires_grad = False

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(lora_params, lr=1e-4, betas=(0.9,0.999), weight_decay=1e-2)

ValueError: optimizer got an empty parameter list

In [ ]:
# 1) Decide what args LoRAAttnProcessor supports in your installed diffusers
sig = inspect.signature(LoRAAttnProcessor.__init__)
supports_hidden = "hidden_size" in sig.parameters
supports_cross  = "cross_attention_dim" in sig.parameters
supports_rank   = "rank" in sig.parameters  # should be True in all recent versions

# 2) Build a LoRA processor for every attention module
RANK = 16
lora_attn_procs = {}

# If your version needs dims, derive them from config once
if supports_hidden:
    hidden_size = transformer.config.attention_head_dim * transformer.config.num_attention_heads
else:
    hidden_size = None
cross_dim = getattr(transformer.config, "cross_attention_dim", None) if supports_cross else None

for name in transformer.attn_processors.keys():
    kwargs = {}
    if supports_hidden: kwargs["hidden_size"] = hidden_size
    if supports_cross:  kwargs["cross_attention_dim"] = cross_dim
    if supports_rank:   kwargs["rank"] = RANK
    lora_attn_procs[name] = LoRAAttnProcessor(**kwargs)

# 3) Install the processors
transformer.set_attn_processor(lora_attn_procs)

# 4) Collect LoRA trainables (a neat wrapper exists in Diffusers)
try:
    lora_layers = AttnProcsLayers(transformer.attn_processors)  # nn.Module wrapper
    lora_params = list(lora_layers.parameters())
except Exception:
    # Fallback: flatten manually
    lora_params = []
    for proc in transformer.attn_processors.values():
        lora_params += list(proc.parameters())

for p in lora_params:
    p.requires_grad = True

print("LoRA trainable params:", sum(p.numel() for p in lora_params))

AttributeError: 'LoRAAttnProcessor' object has no attribute 'parameters'

In [ ]:
import torch.nn.functional as F

@torch.no_grad()
def encode_audio_to_latents(audio_stereo: torch.Tensor):
    """
    audio_stereo: [B, 2, T] in [-1, 1], float32
    returns: latents as expected by DiT
    """
    audio_stereo = audio_stereo.to(dtype=torch.float16, device="cuda")
    latents = vae.encode(audio_stereo).latent_dist.sample()
    # Scale latents (Diffusers convention)
    scaling = getattr(vae.config, "scaling_factor", 0.18215)
    latents = latents * scaling
    return latents

def get_text_embeddings(prompts):
    tokens = tokenizer(list(prompts), padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    with torch.no_grad():
        # For T5, call returns BaseModelOutput with .last_hidden_state at [0]
        out = text_encoder(input_ids=tokens.input_ids, attention_mask=tokens.attention_mask)
        text_emb = out[0] if isinstance(out, (tuple, list)) else out.last_hidden_state
    return text_emb

def noise_pred_loss(batch):
    # Batch tensors
    audio = batch["audio"].to("cuda")  # [B, 2, T]
    prompts = batch["prompt"]
    # Encode to latents
    with torch.no_grad():
        latents = encode_audio_to_latents(audio)  # fp16 on cuda
    # Sample noise & timesteps
    noise = torch.randn_like(latents)
    timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (latents.size(0),), device=latents.device, dtype=torch.long)
    noisy_latents = scheduler.add_noise(latents, noise, timesteps)
    # Text conditioning
    text_emb = get_text_embeddings(prompts)
    # Forward DiT (signature may vary slightly across versions)
    out = transformer(noisy_latents, timesteps, encoder_hidden_states=text_emb)
    noise_pred = out.sample if hasattr(out, "sample") else out[0]
    loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")
    return loss

In [ ]:
from torch.optim import AdamW
from accelerate import Accelerator
from tqdm.auto import tqdm
import time

# Optimizer over LoRA params only
lora_params = [p for n,p in transformer.named_parameters() if p.requires_grad]
optimizer = AdamW(lora_params, lr=1e-4, betas=(0.9,0.999), weight_decay=1e-2)

accelerator = Accelerator(mixed_precision="fp16")
transformer, optimizer, train_loader = accelerator.prepare(transformer, optimizer, train_loader)

MAX_STEPS = 10_000
LOG_EVERY = 50
EVAL_EVERY = 1000

transformer.train()
global_step, run_loss = 0, 0.0

for epoch in range(10**9):
    for batch in train_loader:
        with accelerator.accumulate(transformer):
            loss = noise_pred_loss(batch)
            accelerator.backward(loss)
            accelerator.clip_grad_norm_(lora_params, 1.0)
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
        global_step += 1
        run_loss += float(loss)
        if global_step % LOG_EVERY == 0:
            accelerator.print(f"step {global_step:,} | loss {run_loss/LOG_EVERY:.4f}")
            run_loss = 0.0
        if global_step % EVAL_EVERY == 0:
            transformer.eval()
            accelerator.print("Generating quick validation sample…")
            try:
                wavs = pipe(
                    ["site=MV; outcome=abnormal; murmur=present"],
                    num_inference_steps=120,
                    audio_end_in_s=6.0,
                    num_waveforms_per_prompt=1,
                    generator=torch.Generator(device="cuda").manual_seed(0),
                ).audios
                import soundfile as sf
                out_dir = Path(CIRCOR_ROOT)/"runs"/"lora_sao"
                out_dir.mkdir(parents=True, exist_ok=True)
                sf.write(out_dir/f"val_step{global_step}.wav", wavs[0].T.float().cpu().numpy(), SR_MODEL)
                accelerator.print(f"Saved sample to {out_dir}")
            except Exception as e:
                accelerator.print("Validation generation failed (continuing):", e)
            transformer.train()
        if global_step >= MAX_STEPS:
            break
    if global_step >= MAX_STEPS:
        break

accelerator.print("Training complete.")
# Save LoRA adapter
save_dir = Path(CIRCOR_ROOT)/"runs"/"lora_sao"/"checkpoints"
save_dir.mkdir(parents=True, exist_ok=True)
transformer.save_pretrained(save_dir)  # PEFT save
accelerator.print(f"Saved LoRA checkpoint to {save_dir}")

In [ ]:
# Reload the base pipeline, then load PEFT adapter into the transformer's weights.
from peft import PeftModel

pipe = StableAudioPipeline.from_pretrained(MODEL_REPO, torch_dtype=torch.float16).to("cuda")
lora_dir = Path(CIRCOR_ROOT)/"runs"/"lora_sao"/"checkpoints"

# Attach PEFT adapter to the pipeline's transformer
pipe.transformer = PeftModel.from_pretrained(pipe.transformer, lora_dir)
pipe.transformer.eval()

prompts = [
    "site=MV; outcome=abnormal; murmur=present",
    "site=AV; outcome=normal; murmur=absent",
]
gen = pipe(
    prompts,
    num_inference_steps=200,
    audio_end_in_s=6.0,
    num_waveforms_per_prompt=1,
    generator=torch.Generator(device="cuda").manual_seed(123),
)

import soundfile as sf
out_dir = Path(CIRCOR_ROOT)/"runs"/"lora_sao"/"samples_final"
out_dir.mkdir(parents=True, exist_ok=True)
for i, a in enumerate(gen.audios):
    sf.write(out_dir/f"gen_{i:02d}.wav", a.T.float().cpu().numpy(), SR_MODEL)
print("Saved generations to:", out_dir)